In [99]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pprint
import random
from autogluon.tabular import TabularPredictor

np.random.seed(123)
random.seed(123)

In [100]:
dat1 = pd.read_csv("MMH_15p.csv") #Loading dataset for MMH task

In [101]:
dat2 = pd.read_csv("WLK_13p.csv") #Loading dataset for WLK task

In [102]:
dat1 = dat1.drop(columns=['Unnamed: 0']) #Preprocessing data

In [103]:
dat1.shape

(269, 45)

In [104]:
dat1_train = dat1[:216] #defining training set (80% training, 10% dev, 10% Test)

In [105]:
subsamplesize = 1000
dat1_train = dat1_train.sample(n=subsamplesize, random_state=0, replace = True)

In [106]:
dat1_dev = dat1[216:234]

In [107]:
dat1_test = dat1[235:]
dat1_test_final = dat1_test.drop(columns=['fatiguestate1']) #Dropping the col that's going to be predicted

In [108]:
dat1_train.head() #Checking train data

,subject,task,fatiguestate1,fatiguestate,gender,age,HRR-Mean,HRR-CV,Wrist.jerk.Mean,Wrist.ACC.Mean,...,Wrist.ACC.coefficient.of.variation,Hip.jerk.coefficient.of.variation,Hip.ACC.coefficient.of.variation,Chest.jerk.coefficient.of.variation,Chest.ACC.coefficient.of.variation,Ankle.jerk.coefficient.of.variation,Ankle.ACC.coefficient.of.variation,Hip.yposture.coefficient.of.variation,Chest.yposture.coefficient.of.variation,Ankle.yposture.coefficient.of.variation
172,P10,MMH,1,fatigue,0,20,0.54,4.64,17.056526,8.833597,...,36.53,86.57,53.43,84.78,70.17,106.24,85.24,14.74,16.10,13.40
47,P3,MMH,1,fatigue,0,23,0.29,13.91,13.635994,8.915174,...,58.57,108.77,44.31,99.77,94.31,134.39,90.78,13.66,26.52,10.99
117,P7,MMH,1,fatigue,1,35,0.44,10.47,15.140193,10.000600,...,28.64,83.88,52.04,96.48,58.93,110.08,84.29,10.91,11.49,11.43
192,P11,MMH,1,fatigue,0,25,0.25,15.30,16.779591,7.251594,...,34.62,86.69,53.84,82.02,80.99,115.12,104.23,15.19,23.31,14.09
67,P4,MMH,1,fatigue,1,43,0.34,9.40,13.629112,9.713868,...,32.98,94.98,53.07,85.77,69.54,107.42,84.41,10.50,15.89,12.35


In [109]:
dat1_dev.head() #Checking dev data

,subject,task,fatiguestate1,fatiguestate,gender,age,HRR-Mean,HRR-CV,Wrist.jerk.Mean,Wrist.ACC.Mean,...,Wrist.ACC.coefficient.of.variation,Hip.jerk.coefficient.of.variation,Hip.ACC.coefficient.of.variation,Chest.jerk.coefficient.of.variation,Chest.ACC.coefficient.of.variation,Ankle.jerk.coefficient.of.variation,Ankle.ACC.coefficient.of.variation,Hip.yposture.coefficient.of.variation,Chest.yposture.coefficient.of.variation,Ankle.yposture.coefficient.of.variation
216,P13,MMH,0,notfatigue,0,62,0.26,14.60,11.872341,10.428834,...,21.57,85.52,45.33,86.46,80.53,100.10,74.84,19.00,25.23,10.84
217,P13,MMH,0,notfatigue,0,62,0.26,9.79,12.554526,10.260632,...,21.65,88.17,41.75,87.03,77.36,100.18,75.64,13.87,18.72,10.96
218,P13,MMH,0,notfatigue,0,62,0.25,10.24,12.690959,9.958483,...,22.48,89.44,46.16,86.32,89.04,101.25,75.58,17.63,24.26,10.61
219,P13,MMH,0,notfatigue,0,62,0.23,13.48,11.470788,9.676166,...,29.92,95.14,50.85,89.01,91.30,107.57,79.31,21.93,27.10,10.50
220,P13,MMH,0,notfatigue,0,62,0.24,8.96,12.274799,10.089153,...,24.77,90.74,50.76,87.01,91.18,108.93,81.81,25.13,36.19,11.03


In [110]:
dat1_test_final.head() #Checking test data

,subject,task,fatiguestate,gender,age,HRR-Mean,HRR-CV,Wrist.jerk.Mean,Wrist.ACC.Mean,Hip.jerk.Mean,...,Wrist.ACC.coefficient.of.variation,Hip.jerk.coefficient.of.variation,Hip.ACC.coefficient.of.variation,Chest.jerk.coefficient.of.variation,Chest.ACC.coefficient.of.variation,Ankle.jerk.coefficient.of.variation,Ankle.ACC.coefficient.of.variation,Hip.yposture.coefficient.of.variation,Chest.yposture.coefficient.of.variation,Ankle.yposture.coefficient.of.variation
235,P14,MMH,notfatigue,1,25,0.46,5.52,19.844795,7.425246,27.033785,...,48.57,87.36,43.98,81.78,72.82,107.56,86.12,8.30,31.46,14.75
236,P14,MMH,notfatigue,1,25,0.41,8.88,20.473807,7.546655,24.156795,...,52.42,88.89,42.52,86.23,87.42,104.73,81.43,10.90,41.58,14.05
237,P14,MMH,notfatigue,1,25,0.42,11.29,18.231775,6.807887,28.019898,...,38.94,95.71,47.14,85.18,75.85,105.80,83.23,8.85,30.79,14.56
238,P14,MMH,notfatigue,1,25,0.48,5.04,18.664811,7.693978,19.841645,...,48.78,104.45,46.21,96.13,80.15,122.10,93.95,11.78,42.22,14.83
239,P14,MMH,notfatigue,1,25,0.47,6.76,16.320082,5.954499,27.216956,...,53.78,86.03,46.40,85.29,65.47,106.74,84.33,6.01,26.62,14.55


In [111]:
from autogluon.tabular import TabularPredictor
predictor = TabularPredictor(label='fatiguestate1', path='fat_res')  # specifies folder to store trained models
predictor.fit(dat1_train, hyperparameters='multimodal') #setting multimodal is necessary if the data contains text,numbers,images..

Beginning AutoGluon training ...
AutoGluon will save models to "fat_res/"
AutoGluon Version:  0.6.1
Python Version:     3.9.13
Operating System:   Darwin
Platform Machine:   x86_64
Platform Version:   Darwin Kernel Version 22.1.0: Sun Oct  9 20:15:52 PDT 2022; root:xnu-8792.41.9~2/RELEASE_ARM64_T8112
Train Data Rows:    1000
Train Data Columns: 44
Label Column: fatiguestate1
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [1, 0]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1272.29 MB
	Train Data (Original)  Memory Usage: 0.51 MB (0.0% of avail

In [112]:
predictor.leaderboard(dat1_dev) #Checking the result on dev data

                 model  score_test  score_val  pred_time_test  pred_time_val  fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0             CatBoost    1.000000        1.0        0.003057       0.002288  0.645910                 0.003057                0.002288           0.645910            1       True          1
1  WeightedEnsemble_L2    1.000000        1.0        0.005141       0.002708  0.714229                 0.002084                0.000420           0.068319            2       True          5
2              XGBoost    1.000000        1.0        0.007157       0.003107  0.176847                 0.007157                0.003107           0.176847            1       True          2
3         VowpalWabbit    1.000000        1.0        0.019614       0.022411  0.220741                 0.019614                0.022411           0.220741            1       True          4
4       NeuralNetTorch    0.944444        1.0     

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,CatBoost,1.000000,1.0,0.003057,0.002288,0.645910,0.003057,0.002288,0.645910,1,True,1
1,WeightedEnsemble_L2,1.000000,1.0,0.005141,0.002708,0.714229,0.002084,0.000420,0.068319,2,True,5
2,XGBoost,1.000000,1.0,0.007157,0.003107,0.176847,0.007157,0.003107,0.176847,1,True,2
3,VowpalWabbit,1.000000,1.0,0.019614,0.022411,0.220741,0.019614,0.022411,0.220741,1,True,4
4,NeuralNetTorch,0.944444,1.0,0.012349,0.009027,1.000423,0.012349,0.009027,1.000423,1,True,3


In [49]:
predictions = predictor.predict(dat1_test_final) #Predicting the col on test data

In [83]:
dat1_test_pre = dat1_test['fatiguestate1'] 

In [84]:
perf = predictor.evaluate_predictions(dat1_test_pre, predictions, auxiliary_metrics=True) #Evaluating performance

Evaluation: accuracy on test data: 1.0
Evaluations on test data:
{
    "accuracy": 1.0,
    "balanced_accuracy": 1.0,
    "mcc": 1.0,
    "f1": 1.0,
    "precision": 1.0,
    "recall": 1.0
}


In [66]:
x = pd.DataFrame({'fatiguestate1(Predicted)':predictions.values})
y = dat1_test['fatiguestate1']
y = pd.DataFrame({'fatiguestate1(Original)':y.values})

In [76]:
x.join(y)

,fatiguestate1(Predicted),fatiguestate1(Original)
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,1,1
8,1,1
9,1,1
